ニューラルネットワークモデルの作り方
===================

深層学習では学習における仮説関数にニューラルネットワークを使うことが一般的．

pytorchでももちろん整備されており，[`torch.nn`](https://pytorch.org/docs/stable/nn.html)で用意されているクラス、関数を使って独自のニューラルネットワークを作ることが可能．



In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

クラスの定義
-------------------------
``nn.Module``を継承し、独自のネットワークモデルを定義し、その後ネットワークのレイヤーを ``__init__``で初期化する．

``nn.Module`` を継承した全モジュールは、入力データの順伝搬関数である``forward``関数を持つ。

backward関数を定義する必要はない．nn.moduleで定義してくれてる．

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    torch.cuda.empty_cache()
    torch.cuda.set_device(0)

定義したクラスのインスタンスを作って，GPU上に移す．

In [ ]:
model = Net().cuda()
print(model)

Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


ここで作成したインスタンスを使って実際に学習を進めていく．

モデルレイヤー
-------------------------

作ったモデルを各レイヤーレベルで確認する．


適当な入力を用意する．

MNISTに合わせて，サイズ28x28の3枚の画像からなるミニバッチのサンプルとする．

In [ ]:
input = torch.rand(3,28,28)
print(input.shape)

torch.Size([3, 28, 28])


**nn.Flatten**

[`nn.Flatten`](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)レイヤーで、2次元（28x28）の画像を、1次元の784ピクセルの値へと変換する．

ミニバッチの0次元目は、サンプル番号を示す次元で、この次元は`nn.Flatten`を通しても変化しない（1次元目以降がFlattenされる）．

今回の例であれば3は変化しない．(ミニバッチ数)

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input)
print(flat_image.size())

torch.Size([3, 784])


**nn.Linear** 


[`linear layer`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)は、線形変換を施す．

`linear layer`は重みとバイアスのパラメータを保持している．





In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


**nn.ReLU**

非線形な活性化関数

線形変換だけでなく非線形な変換を施すことがニューラルネットワークを運営していく上で非常に重要になっている．

通常これらの関数は線形変換の後に，ニューラルネットワークの表現力を向上させる役割を持つ．

今回のモデルではReLUを使っている．（一般的にもReLUが多いと思う）

非線形な活性化関数には他にも種類がある．

In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0138,  0.1837,  0.2267,  0.0148, -0.0038, -0.2665,  0.2260,  0.2054,
          0.1403, -0.2230, -0.1733,  0.2614,  0.3257,  0.3238, -0.4651, -0.2401,
          0.0335,  0.1344, -0.0810,  0.1590],
        [ 0.0407,  0.5334,  0.5330, -0.0935, -0.2463, -0.2824, -0.5414,  0.2546,
          0.1329, -0.2451,  0.2114, -0.1061,  0.1594,  0.4361, -0.4799, -0.3238,
         -0.2698, -0.1091,  0.1746,  0.5918],
        [ 0.1618,  0.1593,  0.2756, -0.1153, -0.0890, -0.3470, -0.1498, -0.0178,
          0.0266, -0.3050,  0.3482, -0.0271,  0.2928,  0.2468, -0.2248, -0.3397,
         -0.1738,  0.2635,  0.0956,  0.4242]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.1837, 0.2267, 0.0148, 0.0000, 0.0000, 0.2260, 0.2054, 0.1403,
         0.0000, 0.0000, 0.2614, 0.3257, 0.3238, 0.0000, 0.0000, 0.0335, 0.1344,
         0.0000, 0.1590],
        [0.0407, 0.5334, 0.5330, 0.0000, 0.0000, 0.0000, 0.0000, 0.2546, 0.1329,
         0.0000, 0.2114, 0.0000, 0.1594, 0.4361, 0.00

（ReLU関数はmax{0,x_i}をしてる）

**nn.Sequential**

[``nn.Sequential``](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)は、モジュールを順番に格納する箱のような要素になる．

入力データは``nn.Sequential``に定義された順番に各モジュールを伝搬する．

<br>

例えば以下の実装例のように、``seq_modules``と名付けた、複数のモジュールを束ねたモジュールを簡単に構築でるき．

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

**nn.Softmax**

ニューラルネットワークの最後のlinear layerは`logits` [- ∞, ∞] を出力するがnn.Softmaxモジュールを通すことで採取的な値は[0, 1]の範囲となり、各クラスの確率を返すことになる．


``dim``パラメータは次元を示しており、`dim=1`の次元で和を求めると確率の総和なので1になる．

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

モデルパラメータ
-------------------------

ニューラルネットワークを構成する多くのモジュールは、おのおのパラメータを保持している．

例えば、重みやバイアスがそれに該当する．

これらの値が訓練時に最適化される．


``nn.Module`` を継承することで、モデルオブジェクト内で定義されたすべてのフィールドが自動的に追跡でき、``parameters()`` や ``named_parameters()`` メソッドを使って、モデルの各レイヤーのすべてのパラメータにアクセスできるようになる

これを使ってパラメータの更新や調整を行うことができる．

よく使うのはparameters()

In [ ]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0347,  0.0157, -0.0346,  ...,  0.0337, -0.0033, -0.0101],
        [ 0.0157, -0.0141,  0.0209,  ...,  0.0315,  0.0317, -0.0228]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0303, -0.0311], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0045, -0.0284,  0.0280,  ..., -0.0043, -0.0073,  0.0348],
        [ 0.0430, -0.0284, -0.0266,  ...,  0.0050, -0.0287, -0.0155]],
       device='cuda:0', gra

In [ ]:
for p in model.parameters():
  print(p.shape)

torch.Size([512, 784])
torch.Size([512])
torch.Size([512, 512])
torch.Size([512])
torch.Size([10, 512])
torch.Size([10])
